This notebook updates the tables in [_A Budget-Neutral Universal Basic Income_](https://www.aei.org/wp-content/uploads/2017/05/UBI-working-paper.pdf).

In [1]:
from taxcalc import *
import pandas as pd
import numpy as np

In [2]:
def make_reform(ubi_under18=0, ubi_above18=0):
    tax_reform = {
            'ALD_StudentLoan_hc': {2020: 1.0},
            'ALD_SelfEmploymentTax_hc': {2020: 1.0},
            'ALD_SelfEmp_HealthIns_hc': {2020: 1.0},
            'ALD_KEOGH_SEP_hc': {2020: 1.0},
            'ALD_EarlyWithdraw_hc': {2020: 1.0},
            'ALD_AlimonyPaid_hc': {2020: 1.0},
            'ALD_AlimonyReceived_hc': {2020: 1.0},
            'ALD_Dependents_hc': {2020: 1.0},
            'ALD_EducatorExpenses_hc': {2020: 1.0},
            'ALD_HSADeduction_hc': {2020: 1.0},
            'ALD_IRAContributions_hc': {2020: 1.0},
            'ALD_DomesticProduction_hc': {2020: 1.0},
            'ALD_Tuition_hc': {2020: 1.0},
            'ALD_BusinessLosses_c': {2020: [0.0, 0.0, 0.0, 0.0, 0.0]},
            'CR_RetirementSavings_hc': {2020: 1.0},
            'CR_ForeignTax_hc': {2020: 1.0},
            'CR_ResidentialEnergy_hc': {2020: 1.0},
            'CR_GeneralBusiness_hc': {2020: 1.0},
            'CR_MinimumTax_hc': {2020: 1.0},
            'CR_AmOppRefundable_hc': {2020: 1.0},
            'CR_AmOppNonRefundable_hc': {2020: 1.0},
            'CR_SchR_hc': {2020: 1.0},
            'CR_OtherCredits_hc': {2020: 1.0},
            'CR_Education_hc': {2020: 1.0},
            'II_em': {2020: 0.0},
            'STD': {2020: [0.0, 0.0, 0.0, 0.0, 0.0]},
            'STD_Aged': {2020: [0.0, 0.0, 0.0, 0.0, 0.0]},
            'STD_Dep': {2020: 0.0},
            'ID_c': {2020: [0, 0, 0, 0, 0]},
            'CDCC_c': {2020: 0.0},
            'CTC_c': {2020: 0.0},
            'ACTC_c': {2020: 0.0},
            'ODC_c': {2020: 0.0},
            'EITC_c': {2020: [0.0, 0.0, 0.0, 0.0]},
            'PT_qbid_rt': {2020: 0.0},
            'LLC_Expense_c': {2020: 0.0},
            'ETC_pe_Single': {2020: 0.0},
            'ETC_pe_Married': {2020: 0.0},
            'BEN_housing_repeal': {2020: True},
            'BEN_ssi_repeal': {2020: True},
            'BEN_snap_repeal': {2020: True},
            'BEN_tanf_repeal': {2020: True},
            'BEN_vet_repeal': {2020: True},
            'BEN_wic_repeal': {2020: True},
            'BEN_mcare_repeal': {2020: True},
            'BEN_mcaid_repeal': {2020: True},
            'BEN_oasdi_repeal': {2020: True},
            'BEN_ui_repeal': {2020: True},
            'BEN_other_repeal': {2020: True},
            'UBI_u18': {2020: ubi_under18},
            'UBI_1820': {2020: ubi_above18},
            'UBI_21': {2020: ubi_above18}
    }
    pol = Policy()
    pol.implement_reform(tax_reform)
    return pol

In [3]:
# initiate base CPS calc with welfare multiples
welfare_multiples = {
        'BEN_vet_value': {2020: 0.95},
        'BEN_mcare_value': {2020: 0.75},
        'BEN_mcaid_value': {2020: 0.30}
}

consumption = Consumption()
consumption.update_consumption(welfare_multiples)

cps_calc = Calculator(records=Records.cps_constructor(), policy=Policy(), consumption=consumption)
cps_calc.advance_to_year(2020)
cps_calc.calc_all()

In [4]:
cps_calc.weighted_total('mcare_ben') / 1e9

805.2829100726923

In [5]:
# total benefits from cps
cps_rev = cps_calc.weighted_total('benefit_cost_total')
cps_rev / 1e9

3117.9817596416115

In [6]:
# Base PUF calculator
recs = Records()
calc = Calculator(records=recs, policy=Policy(), verbose=False)
calc.advance_to_year(2020)
calc.calc_all()

In [7]:
# calculate revenue gained from tax reform with PUF calculator (no UBI is implemented)
calc_tax_rev = Calculator(records=recs, policy=make_reform())
calc_tax_rev.advance_to_year(2020)
calc_tax_rev.calc_all()
puf_tax_rev = calc_tax_rev.weighted_total('combined') - calc.weighted_total('combined')
puf_tax_rev / 1e9

CTC_c was redefined in release 1.0.0



767.6330331449511

In [8]:
# total benefits from repealing benefits (cps_rev) and tax reform (puf_tax_rev)
revenue = cps_rev + puf_tax_rev
revenue / 1e9

3885.6147927865627

In [9]:
# Number above and below 18
u18 = (calc.array('nu18') * calc.array('s006')).sum()
abv18 = ((calc.array('n1820') + calc.array('n21')) * calc.array('s006')).sum()
u18, abv18

(85397017.52000001, 256113916.59999993)

In [10]:
# function to calculate budget-neutral UBI amounts given tax reform and benefits repeal
def ubi_amt(revenue, u18, abv18):
    ubi_18 = revenue / ((u18 * 0.5) + abv18)
    ubi_u18 = ubi_18 * 0.5
    total_ubi = (ubi_18 * abv18) + (ubi_u18 * u18)
    return float(ubi_u18), float(ubi_18) 

# UBI per person amounts, not accounting for revenue raised by taxing UBI
ubiu18_untaxed, ubi18_untaxed = ubi_amt(revenue, u18, abv18)
ubiu18_untaxed, ubi18_untaxed

(6501.7624151761665, 13003.524830352333)

In [11]:
def ubi_finder(ubi_u18, ubi_18, revenue):
    # Build a calculator with the specified UBI levels
    pol_finder = make_reform(ubi_u18, ubi_18)
    calc_finder = Calculator(records=Records(), policy=pol_finder, verbose=False)
#     calc_finder.records.e02400 = np.zeros(len(calc_finder.records.e02400))
    calc_finder.advance_to_year(2020)
    calc_finder.calc_all()
    # Check if UBI is greater or less than the additional revenue
    # Revenue from tax reform
    ubi_tax_rev = calc_finder.weighted_total('combined') - calc_tax_rev.weighted_total('combined')
    total_rev = ubi_tax_rev + revenue
    ubi = calc_finder.weighted_total('ubi')
    diff = ubi - total_rev
    return diff, ubi_tax_rev

In [12]:
# Find UBI after accounting for UBI tax revenue
diff = 9e99
ubi_tax_rev = 0
prev_ubi_tax_rev = 0
while abs(diff) >= 100:
    ubi_u18, ubi_18 = ubi_amt(revenue + ubi_tax_rev, u18, abv18)
    diff, ubi_tax_rev = ubi_finder(ubi_u18, ubi_18, 
                                   revenue=revenue)
    if diff > 0:
        ubi_tax_rev = prev_ubi_tax_rev * 0.5
    prev_ubi_tax_rev = ubi_tax_rev
ubi_u18, ubi_18

CTC_c was redefined in release 1.0.0

CTC_c was redefined in release 1.0.0

CTC_c was redefined in release 1.0.0

CTC_c was redefined in release 1.0.0

CTC_c was redefined in release 1.0.0

CTC_c was redefined in release 1.0.0

CTC_c was redefined in release 1.0.0

CTC_c was redefined in release 1.0.0

CTC_c was redefined in release 1.0.0

CTC_c was redefined in release 1.0.0

CTC_c was redefined in release 1.0.0

CTC_c was redefined in release 1.0.0

CTC_c was redefined in release 1.0.0

CTC_c was redefined in release 1.0.0

CTC_c was redefined in release 1.0.0



(7922.599098105521, 15845.198196211042)

In [13]:
# CPS and PUF calculators with tax reform + UBI

pol_reform_ubi = make_reform(ubi_u18, ubi_18)

calc_ubi = Calculator(records=Records(), policy=pol_reform_ubi, verbose=False)
calc_ubi.advance_to_year(2020)
calc_ubi.calc_all()

cps_calc_ubi = Calculator(records=Records.cps_constructor(), policy=pol_reform_ubi, consumption=consumption, verbose=False)
cps_calc_ubi.advance_to_year(2020)
cps_calc_ubi.calc_all()

CTC_c was redefined in release 1.0.0



In [14]:
var_list = ['benefit_cost_total', 'benefit_value_total',
            'c00100', 'combined', 's006', 'ubi', 'nu18', 'n1820', 'n21', 'elderly_dependents', 'age_head', 'age_spouse', 'e00200']

base_puf_data = calc.dataframe(var_list)
ubi_puf_data = calc_ubi.dataframe(var_list)

base_cps_data = cps_calc.dataframe(var_list)
ubi_cps_data = cps_calc_ubi.dataframe(var_list)

In [15]:
# create a table from the CPS data
cps_table_data = pd.DataFrame()
cps_table_data['e00200'] = base_cps_data['e00200']
cps_table_data['s006'] = base_cps_data['s006']
cps_table_data['ppl'] = ubi_cps_data['nu18'] + ubi_cps_data['n1820'] + ubi_cps_data['n21']
cps_table_data['ubi'] = ubi_cps_data['ubi']
cps_table_data['ubi/person'] = cps_table_data['ubi'] / cps_table_data['ppl']
cps_table_data['ben_change'] = ubi_cps_data['benefit_value_total'] - base_cps_data['benefit_value_total']
cps_table_data['plus_65'] = np.where((base_cps_data['elderly_dependents'] >= 1) |
                                     (base_cps_data['age_head'] >= 65) |
                                     (base_cps_data['age_spouse'] >= 65), 1, 0)

In [16]:
# create a table from the puf data
puf_table_data = pd.DataFrame()
puf_table_data['e00200'] = base_puf_data['e00200']
puf_table_data['s006'] = base_puf_data['s006']
puf_table_data['ppl'] = ubi_puf_data['nu18'] + ubi_puf_data['n1820'] + ubi_puf_data['n21']
puf_table_data['tax_change'] = ubi_puf_data['combined'] - base_puf_data['combined']
puf_table_data['plus_65'] = np.where((base_puf_data['elderly_dependents'] >= 1) |
                                     (base_puf_data['age_head'] >= 65) |
                                     (base_puf_data['age_spouse'] >= 65), 1, 0)

In [17]:
# create subset dataframe for filing units under 65 years old
cps_table_data_u65 = cps_table_data.loc[cps_table_data['plus_65'] == 0]
puf_table_data_u65 = puf_table_data.loc[puf_table_data['plus_65'] == 0]

# create subset dataframe for filing units with at least one 65 year old
cps_table_data_o65 = cps_table_data.loc[cps_table_data['plus_65'] == 1]
puf_table_data_o65 = puf_table_data.loc[puf_table_data['plus_65'] == 1]

In [18]:
income_bins = [0, 9999, 19999, 29999, 39999, 49999,
               74999, 99999, 199999, 1000000, 9e99]
floors = [0, 9999, 19999, 29999, 39999, 49999,
               74999, 99999, 199999, 1000000]

In [19]:
def table_values(data, var):
    """
    Function to return the values for each table
    """
    val = data.groupby('bins', as_index=False).apply(weighted_mean, var)
    return val

def cps_table(data, income_measure, bins=None):
    """
    Function to create a table
    Parameters
    ----------
    data: DataFrame containing data from Tax-Calculator
    income_measure: income used for binning
    bins: number of desired bins
    """

    # add bin labels
    data['bins'] = pd.cut(data[income_measure], bins, include_lowest=True)
    decile_floor = pd.Series(floors)
    
    # Create DataFrame with all relevant information
    info = pd.DataFrame()
    if income_measure == 'e00200':
        income_str = 'Wage and Salary Floor (Thousands)'
    elif income_measure == 'c00100':
        income_str = 'AGI Floor (Thousands)'

    info[income_str] = (decile_floor / 1000).apply('{:,.0f}'.format)
    info['Avg UBI Per Person'] = table_values(data, 'ubi/person')
    info['Avg UBI Per Tax Unit'] = table_values(data, 'ubi')
    info['Avg Benefits Change'] = table_values(data, 'ben_change')

    return info

def puf_table(data, income_measure, bins=None):
    """
    Function to create a table
    Parameters
    ----------
    data: DataFrame containing data from Tax-Calculator
    income_measure: income used for binning
    bins: number of desired bins
    """

    # add bin labels
    data['bins'] = pd.cut(data[income_measure], bins, include_lowest=True)
    decile_floor = pd.Series(floors)
    
    info = pd.DataFrame()
    if income_measure == 'e00200':
        income_str = 'Wage and Salary Floor (Thousands)'
    elif income_measure == 'c00100':
        income_str = 'AGI Floor (Thousands)'
        
    info[income_str] = (decile_floor / 1000).apply('{:,.0f}'.format)
    
    # total tax units
    units = data.groupby('bins', as_index=False).sum()['s006']
    info['Tax Units (m)'] = (units / 1000000).apply('{:,.2f}'.format)
    
    # average tax unit size
    avg_size = table_values(data, 'ppl')
    info['Avg Tax Unit Size'] = avg_size.apply('{:,.2f}'.format)
    info['Avg Tax Change'] = table_values(data, 'tax_change')

    return info

In [20]:
puf_df_all = puf_table(puf_table_data, 'e00200', income_bins)
cps_df_all = cps_table(cps_table_data, 'e00200', income_bins)

puf_df_u65 = puf_table(puf_table_data_u65, 'e00200', income_bins)
cps_df_u65 = cps_table(cps_table_data_u65, 'e00200', income_bins)

puf_df_o65 = puf_table(puf_table_data_o65, 'e00200', income_bins)
cps_df_o65 = cps_table(cps_table_data_o65, 'e00200', income_bins)

/Users/petermetz/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:54: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/Users/petermetz/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [21]:
def make_final_table(puf_data, cps_data):
    final_table = puf_data.merge(cps_data, on='Wage and Salary Floor (Thousands)')
    final_table['Avg Combined Change'] = (final_table[['Avg UBI Per Tax Unit', 'Avg Benefits Change']].sum(axis=1) -
                                      final_table['Avg Tax Change'])
    return final_table
#     return final_table.style.format('${:,.2f}', subset=['Avg Tax Change', 'Avg UBI Per Tax Unit', 'Avg UBI Per Person',
#                                             'Avg Benefits Change',
#                                             'Avg Combined Change'])

In [22]:
# reform results for all tax units
final_table_all = make_final_table(puf_df_all, cps_df_all)
final_table_all

,Wage and Salary Floor (Thousands),Tax Units (m),Avg Tax Unit Size,Avg Tax Change,Avg UBI Per Person,Avg UBI Per Tax Unit,Avg Benefits Change,Avg Combined Change
0,0,73.82,1.54,4936.600786,15051.774155,23806.222840,-31185.129582,-12315.507528
1,10,15.43,1.73,6766.689397,14596.190781,25068.152867,-13021.593446,5279.870024
2,20,13.00,1.91,8118.485061,14630.294107,25382.971368,-9509.275680,7755.210627
3,30,11.53,1.98,8779.372954,14633.169675,26164.611114,-8179.794009,9205.444151
4,40,10.14,2.01,9535.013991,14608.622131,27426.162682,-7632.665066,10258.483625
5,50,17.38,2.06,10716.245510,14600.862804,28785.839194,-7162.803849,10906.789835
6,75,10.52,2.31,13161.531671,14447.755037,32141.937928,-6241.641207,12738.765050
7,100,18.77,2.76,17127.088683,14202.855978,36418.948880,-5734.479348,13557.380848
8,200,6.69,3.02,26008.880694,14114.392986,38893.841042,-4674.602398,8210.357950
9,"1,000",0.24,3.23,101249.008952,14062.593001,37984.620209,-5162.211065,-68426.599809


In [23]:
# reform results for tax units with individuals 65+ years old
final_table_o65 = make_final_table(puf_df_o65, cps_df_o65)
final_table_o65

,Wage and Salary Floor (Thousands),Tax Units (m),Avg Tax Unit Size,Avg Tax Change,Avg UBI Per Person,Avg UBI Per Tax Unit,Avg Benefits Change,Avg Combined Change
0,0,34.72,1.50,4436.546539,15683.243637,24043.084044,-44545.614981,-24939.077476
1,10,1.94,1.78,6509.741326,15647.437210,26854.345050,-42747.404492,-22402.800768
2,20,1.25,2.00,7766.315392,15579.127568,28289.522733,-38659.301995,-18136.094655
3,30,1.06,2.03,7730.377723,15563.398655,28310.662797,-36817.571434,-16237.286361
4,40,0.87,2.13,8466.415925,15534.958168,29690.959797,-38440.418046,-17215.874175
5,50,1.51,2.15,9522.422433,15544.496953,29894.574977,-37691.323429,-17319.170885
6,75,0.92,2.16,11230.100015,15545.225384,30796.108559,-36522.956652,-16956.948108
7,100,1.14,2.26,14085.411151,15503.187218,32536.991845,-36105.203923,-17653.623228
8,200,0.47,2.36,25160.311649,15443.605944,34163.055405,-31706.989992,-22704.246237
9,"1,000",0.02,2.21,121506.466149,15407.372152,31711.122753,-31286.052265,-121081.395661


In [24]:
# reform results for tax units with individuals under 65 years old
final_table_u65 = make_final_table(puf_df_u65, cps_df_u65)
final_table_u65

,Wage and Salary Floor (Thousands),Tax Units (m),Avg Tax Unit Size,Avg Tax Change,Avg UBI Per Person,Avg UBI Per Tax Unit,Avg Benefits Change,Avg Combined Change
0,0,39.10,1.58,5380.595576,14411.987011,23566.241721,-17648.661015,536.985130
1,10,13.49,1.72,6803.671950,14440.457636,24803.543809,-8617.969394,9381.902465
2,20,11.75,1.90,8155.926140,14531.451185,25080.186905,-6472.627107,10451.633658
3,30,10.47,1.97,8885.589951,14537.930823,25944.893707,-5247.796489,11811.507267
4,40,9.27,2.00,9635.040556,14516.237680,27200.292125,-4560.176022,13005.075547
5,50,15.87,2.06,10829.761203,14493.172712,28659.307256,-3678.806305,14150.739747
6,75,9.59,2.33,13347.339564,14330.460561,32285.776303,-3005.261244,15933.175495
7,100,17.63,2.79,17323.085142,14058.767732,36849.104185,-2369.133427,17156.885616
8,200,6.22,3.06,26073.153854,13980.078429,39371.878346,-1943.028683,11355.695809
9,"1,000",0.22,3.32,99497.648015,13910.172612,38695.672968,-2201.274283,-63003.249331


In [25]:
from bokeh.io import show, output_notebook
from bokeh.plotting import figure
from bokeh.models import ColumnDataSource, HoverTool, NumeralTickFormatter, Panel, Tabs, LabelSet

def make_graph_data(input_data):
    graph_data = pd.DataFrame()
#     graph_data['Wage and Salary Floor (Thousands)'] = input_data['Wage and Salary Floor (Thousands)']
    graph_data['Wage and Salary Floor (Thousands)'] = np.array(['$0 - $10', '$10 - $20', '$20 - $30', '$30 - $40', '$40 - $50','$50 - $75','$75 - $100','$100 - $200','$200 - $1,000','$1,000+'])
    graph_data['Additional Taxes Paid'] = input_data['Avg Tax Change'] * -1
    graph_data['Benefits/Transfers Lost'] = input_data['Avg Benefits Change']
    graph_data['UBI'] = input_data['Avg UBI Per Tax Unit']
    graph_data['Combined_Change'] = input_data['Avg Combined Change']
    return graph_data

graph_data_all = make_graph_data(final_table_all)
graph_data_u65 = make_graph_data(final_table_u65)
graph_data_o65 = make_graph_data(final_table_o65)

buckets = graph_data_all['Wage and Salary Floor (Thousands)'].to_list()

bar1 = ['UBI']
bar2 = ['Additional Taxes Paid', 'Benefits/Transfers Lost']
was = 'Wage and Salary Floor (Thousands)'

source_all = ColumnDataSource(graph_data_all)
source_u65 = ColumnDataSource(graph_data_u65)
source_o65 = ColumnDataSource(graph_data_o65)


In [26]:
output_notebook()

Loading BokehJS ...

In [29]:
def make_figures(source):
    f = figure(x_range=buckets, plot_height=400, plot_width=800, toolbar_location=None)
    f.xgrid.grid_line_color = None
    
    f.yaxis.axis_label = "After Tax Income + Benefits"
    f.yaxis.major_label_orientation = "horizontal"
    f.yaxis.formatter = NumeralTickFormatter(format='($0,0)')
    f.yaxis.minor_tick_line_color = None
    
    f.xaxis.axis_label = "Tax Unit Wage and Salary Bucket (Thousands)"
    f.xaxis.minor_tick_line_color = 'black'
    f.xaxis.major_tick_line_color = None

    f.vbar_stack(bar1, x=was, width=0.9, color='#79D151', source=source, legend_label=bar1, alpha=0.8, name=bar1)  
    f.vbar_stack(bar2, x=was, width=0.9, color=['#22A784','#29788E'], source=source, legend_label=bar2, alpha=0.8, name=bar2)
    f.line(x=was, y='Combined_Change', source=source, legend_label='Combined Change', line_width=2, color='#404387')
    
    ht = HoverTool(tooltips="$name: @$name{($:,)}", names=['UBI', 'Additional Taxes Paid', 'Benefits/Transfers Lost'])
    f.tools.append(ht)
    
    f.legend.location = 'bottom_left'
    return f

f_all = make_figures(source_all)
f_u65 = make_figures(source_u65)
f_o65 = make_figures(source_o65)

tab1 = Panel(child=f_all, title="All Tax Units")
tab2 = Panel(child=f_u65, title="Tax Units - All Individuals Under 65")
tab3 = Panel(child=f_o65, title="Tax Units - At Least one Individual Over 65")

tabs = Tabs(tabs=[tab1, tab2, tab3])
show(tabs)